In [1]:
import mip

In [2]:
def degree_of_neighborhoods(arr_1d):
    """
    [1 1 1 0 0] => 2
    [1 0 1 0 1] => 0
    [0 1 1 0 0] => 1
    [0 1 1 0 1] => 1
    [1 1 0 1 0] => 1
    """
    reward_candidates = [0]
    is_prev_one = False
    
    for v in arr_1d:
        if v == 0:
            is_prev_one = False
            if reward_candidates[-1] > 0:
                reward_candidates.append(0)
            continue
        elif is_prev_one:
            is_prev_one = True
            reward_candidates[-1] += 1
        else:
            is_prev_one = True
    
    return max(reward_candidates)

for test_arr, expected in [
    ([1, 1, 1, 0, 0], 2),
    ([1, 0, 1, 0, 1], 0),
    ([0, 1, 1, 0, 0], 1),
    ([0, 1, 1, 0, 1], 1),
    ([1, 1, 0, 1, 0], 1),
]:
    result = degree_of_neighborhoods(test_arr)
    if result != expected:
        print(f"Tested {test_arr} => {expected} but got {result}")

In [3]:
def count_adjacent_ones(arr):
    return sum(arr[i] == 1 and arr[i+1] == 1 for i in range(len(arr)-1))

for test_arr, expected in [
    ([1, 1, 1, 0, 0], 2),
    ([1, 0, 1, 0, 1], 0),
    ([0, 1, 1, 0, 0], 1),
    ([0, 1, 1, 0, 1], 1),
    ([1, 1, 0, 1, 0], 1),
]:
    result = count_adjacent_ones(test_arr)
    if result != expected:
        print(f"Tested {test_arr} => {expected} but got {result}")

In [4]:
hoge = [1, 1, 1, 0, 0]
fuga = [1, 0, 1, 0, 1]

def union(a,b):
    return [max(a[i], b[i]) for i in range(len(a))]

def product(a,b):
    return [min(a[i], b[i]) for i in range(len(a))]

print(union(hoge, fuga))
print(product(hoge, fuga))

[1, 1, 1, 0, 1]
[1, 0, 1, 0, 0]


In [8]:
m = mip.Model()

# Variables
x = m.add_var_tensor((5,), "x", var_type=mip.BINARY)
y = m.add_var_tensor((5,), "y", var_type=mip.BINARY)

# Constraints: the conditions that must be met

# 各人のスケジュールに合わせる。
# xの3,4はシフトに入れない
m += x[3] == 0
m += x[4] == 0
# yの0,1はシフトに入れない
m += y[0] == 0
m += y[1] == 0

# シフトは埋められなければならない
# かつ、同じ枠には2人以上入れない
for i in range(len(x)):
    m += x[i] + y[i] == 1

# Objective: the value to be optimized
# m.objective = mip.maximize(degree_of_neighborhoods(x))
m.objective = mip.maximize(mip.xsum(x[i] == 1 and x[i+1] == 1 for i in range(len(x)-1)))
m.objective = mip.maximize(mip.xsum(y[i] == 1 and y[i+1] == 1 for i in range(len(y)-1)))

In [9]:
m.optimize()

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-9) rows, 0 (-10) columns and 0 (-14) elements
Clp0000I Optimal - objective value 3
Coin0511I After Postsolve, objective 3, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 3 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0002I 4 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



<OptimizationStatus.OPTIMAL: 0>

In [10]:
for v in x:
    print(v.x)

for v in y:
    print(v.x)

1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
